In [4]:
import distances
from sklearn.metrics.pairwise import pairwise_distances
import numpy

### Load example scikit-learn dataset
http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn.datasets.fetch_covtype
http://archive.ics.uci.edu/ml/datasets/Covertype
#### Predicting forest cover type 

In [2]:
from sklearn.datasets import fetch_covtype
sklearn_data = fetch_covtype()
print(sklearn_data.data.shape)

(581012, 54)


In [21]:
nof_objects = sklearn_data.data.shape[0]
nof_objects_use = 100000 # Use only a subset of the objects in this dataset
nof_trees = 100 # Number of trees used to calculate the similarity
nof_test_objects = 100 # Test objects for nearest neighbor classification

In [11]:
X_use = sklearn_data.data[numpy.random.choice(numpy.arange(nof_objects), nof_objects_use)]

### Calculate a matrix containing all the nodes each object goes through in each tree

In [12]:
%%time
tree_path = distances.ERT_tree_path_list(X=X_use,n_estimators=nof_trees)

CPU times: user 18min 32s, sys: 12.8 s, total: 18min 44s
Wall time: 19min 9s


In [13]:
# This is the path on tree tree_index for object object_index. 
# The values are the numbering of the nodes. 
#We use this matrix to count the number of splits two objects went through toghether in each tree
tree_index = 0
object_index = 5
tree_path[tree_index][object_index]

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int32)

## ERT similarities
#### We calculate the similarities and use them for nearest neighbor classification. This is compared to nearest neighbor classification with manhatten distances, and to random prediction.

In [22]:
values = numpy.zeros(nof_test_objects)
prediction_ert = numpy.zeros(nof_test_objects)
prediction_manhatten = numpy.zeros(nof_test_objects)
prediction_random = numpy.zeros(nof_test_objects)

test_objects = numpy.random.choice(numpy.arange(nof_objects_use), nof_test_objects)

In [23]:
manhattan_dis = pairwise_distances(X = X_use[test_objects], Y = X_use, metric='manhattan')
manhattan_ngbs = numpy.argsort(manhattan_dis, axis = 1)[:,1]

In [ ]:
import tqdm
for i, global_i in tqdm.tqdm(enumerate(test_objects)):
    
    # This function calculated the similarities between one objct and all the rest
    ert_sims = distances.ERT_similarity(idx=global_i,tree_path=tree_path) 
    
    # After having the similarities we can get the nearest neighbor
    ert_nn_idx = ert_sims.argsort()[::-1][1]
    
    # This is the manhatten distance nearest neighbor
    manhatten_nn_idx = manhattan_ngbs[i]
    
    # Keep the actual value (comes with the dataset we loaded)
    values[i] = sklearn_data.target[global_i]
    # ERT nearest neighbor prediction
    prediction_ert[i] = sklearn_data.target[ert_nn_idx]
    # Manhatten nearest neighbor prediction
    prediction_manhatten[i] = sklearn_data.target[manhatten_nn_idx]
    # Random prediction, drawn from the set of target values
    prediction_random[i] = sklearn_data.target[numpy.random.choice(numpy.arange(nof_objects))]


0it [00:00, ?it/s]
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/Users/itamar/anaconda2/envs/avocado/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/Users/itamar/anaconda2/envs/avocado/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/itamar/anaconda2/envs/avocado/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

9it [03:08, 20.92s/it]

In [ ]:
print(' ERT:      ', numpy.sum(val == pred_ert      ), '\n',
      'Manhatten:', numpy.sum(val == pred_manhatten), '\n',
      'Random:   ', numpy.sum(val == random_pred   ))